# DashStyle Fusion with OpenAI
#### Apply the style of one ArcGIS Dashboard to other ArcGIS Dashboards
created by Niklas Köhn, Esri Deutschland, 2024

In [1]:
from arcgis.gis import GIS
import getpass

# user input username + password
username = input("Username: ")
password = getpass.getpass(prompt='Password: ', stream=None) # hide password

# login to ArcGIS Online or ArcGIS Enterprise
gis = GIS("https://www.arcgis.com", username, password)
print("Successfully logged in as: " + gis.properties.user.username)

Username: esride-nik
Password: ········
Successfully logged in as: esride-nik


### Search for dashboards

In [2]:
# user select to search org or own account
print("1. Search in the organization")
print("2. Search in my account")
choice = input("Enter your choice: ")

# search for dashboards in the org or own account
outside_org_choice = True
if choice == "1":
    outside_org_choice = False

search = input("Enter search term (empty string to search for all dashboards): ")
items = gis.content.search(query=search, item_type="Dashboard", max_items=100, outside_org=outside_org_choice)
print("Found " + str(len(items)) + " dashboards")

for item in items:
    print(item.title + " (" + item.id + ")")

1. Search in the organization
2. Search in my account
Enter your choice: 1
Enter search term (empty string to search for all dashboards): verkehr
Found 26 dashboards
Köln Verkehr Test SVHA (f22ec93ff4cb4114875e548341ea9df5)
STA LGV HH - Verkehrsdaten Rad 5 Min - WIP (64f0f27d96864f1cb314ee60f0abd6cf)
Pol RLP Verkehrsunfälle (01ad780b11854a9bbe189297accba752)
Verkehrskameras (93f3ec140c14454ab8c3e9dc80896acb)
HERE Verkehrsdatenanalyse (742ffe68052645bdaf449deb9370eb92)
Umweltverbund in Kölner Stadtteilen (da4a6f40373646688fcd45fec3b89816)
Verkehrsmonitoring Köln v1.2 - Kopi (8091a9d0bb514484a11b68a736c36b30)
Verkehrs-Dashboard Bonn - ALT (5123bec941b64264b5d5fa94461fafab)
Strafzettel in Köln (Dezember 2016) (776b4f62414247588bede1b393ba76c5)
Pol RLP Verkehrsunfälle (kopieren) (a6c6207c4df840f0990d5344605e08ed)
Ordnungswidrigkeiten mit FS-Kennzeichen in Köln im Jahr 2017 (5259b95562a84686ac9bc7a27c23ce29)
HERE Verkehrsdatenanalyse-Ingo (3dd7a399262a44b6b2dafdc9c0c8737f)
dashboard_vod_ver

### Copy all dashboards to folder in own account

In [3]:
import datetime
import sys

# create folder in own account named "dashboards_<date>_time"
folder_name = "dashboards_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
folder = gis.content.create_folder(folder_name)

# copy all dashboards to the folder
copied_items = []
for item in items:
    # use arcgis python api to copy dashboard item. folder parameter does not work, everything ends up in root :(
    copied_item = item.copy_item(folder=folder_name)
    copied_items.append(copied_item)
    print ("Dashboard " + copied_item.title + " (" + copied_item.id + ")" + " copied.")

# workaround because folder parameter doesn't work: move items. if this fails, delete copies.
moved_items = []
deleted_items = []
for item in copied_items:
    objMoved = item.move(folder=folder_name)  #folder not found for given owner message if it doesnt move it.
    moved_items.append(item)
    if (objMoved is None) or \
            (objMoved is not None and 'success' in objMoved and not objMoved['success']):
        deleted_items.append(item)
        item.delete(force=False, dry_run=False)
        sys.exit(
            "ERROR: Unable to move the item with ItemID {0} to the folder, {1}. ".format(
                id, folder_name))
    
for item in moved_items:
    print ("Dashboard " + item.title + " (" + item.id + ")" + " moved to folder " + folder_name + ".")
print (str(len(moved_items)) + " dashboards copied in total.")

Dashboard Köln Verkehr Test SVHA (0480577e76ac445fa0c933b936518fc7) copied.
Dashboard STA LGV HH - Verkehrsdaten Rad 5 Min - WIP (8cd84310ce3b45f2abde10c3c9b66896) copied.
Dashboard Pol RLP Verkehrsunfälle (98fd29d5adb141308c4fdbf6e12c63a6) copied.
Dashboard Verkehrskameras (513738ffd69c4cd5a558d92e2ae0647c) copied.
Dashboard HERE Verkehrsdatenanalyse (700f9f63c6c447848f3e8de02ea08973) copied.
Dashboard Umweltverbund in Kölner Stadtteilen (a933e7ab1826492da18c3e4eef265b05) copied.
Dashboard Verkehrsmonitoring Köln v1.2 - Kopi (2d3ba7964acc415fbd2ad3a72be9f4ce) copied.
Dashboard Verkehrs-Dashboard Bonn - ALT (61bcf5ea3fec470d9a5d09d88862e1e7) copied.
Dashboard Strafzettel in Köln (Dezember 2016) (e942fc53df12478487c2b6ceb0810f6b) copied.
Dashboard Pol RLP Verkehrsunfälle (kopieren) (aadf87cedac142b69001ac8146d563b7) copied.
Dashboard Ordnungswidrigkeiten mit FS-Kennzeichen in Köln im Jahr 2017 (4fec7915136449eb869da470cbc1dac4) copied.
Dashboard HERE Verkehrsdatenanalyse-Ingo (df91ba9d4

### Select template

Select template dashboard by item ID and get the JSON

In [36]:
# user input dashboard id to use as template
template_id = input("Enter dashboard id to use as template: ")

# filter moved items to find template dashboard
template_item = None
for item in moved_items:
    if item.id == template_id:
        template_item = item
        break
print("template_item", template_item)

# get template dashboard JSON
template_json = template_item.get_data()

Enter dashboard id to use as template: 6a13139b84ae4a978748356fa81b4aab
template_item <Item title:"Verkehrsdashboard Bonn" type:Dashboard owner:esride-nik>


In [37]:
# get header, theme and themeOverrides manually from template_json

header = None

try:
    # copy header section from template_json
    if template_json["version"] > 57:
    # TODO: check since when we have a "desktopView"
        header = template_json["desktopView"]["header"]
    else:
        header = template_json["header"]
except:
    print ('no header found')

try:
    # copy theme property from template_json
    theme = template_json["theme"]
except:
    print ('no theme found')
    
try:
    # copy themeOverrides from template_json
    themeOverrides = template_json["themeOverrides"]
except:
    print ('no themeOverrides found')

print (header, "\n\n ... \n\n", theme, "\n\n ... \n\n", themeOverrides)

no header found
None 

 ... 

 light 

 ... 

 {}


In [6]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0


In [87]:
# Search all widgets

import sys

# skip this cell if we already have a header
if header is not None:
    sys.exit('Skipping OpenAI attempt because we already have a header.')

unwanted_widget_types = ['mapWidget', 'indicatorWidget', 'listWidget', 'embeddedContentWidget', 'serialChartWidget', 'pieChartWidget', 'gaugeWidget', 'legendWidget', 'featureDetailsWidget', 'queryWidget']
wanted_widget_types = ['richTextWidget'] #, 'detailsWidget', 'descriptionWidget']
possible_header_widgets = []

for item in moved_items:
    item_json = item.get_data()
    
    widgets = None
    if 'widgets' in item_json:
        widgets = item_json['widgets']
    if 'desktopView' in item_json:
        widgets = item_json['desktopView']['widgets']
    if widgets == None:
        print ('Skipping item because it has no widgets.', item_json)
        continue
    
    filtered_widgets = list(map(lambda x: {'itemID': item.id, 'widget': x}, filter(lambda x: x["type"] in wanted_widget_types, widgets)))
#     filtered_widgets = filter(lambda x: x["type"] not in unwanted_widget_types, widgets)
    possible_header_widgets.extend(filtered_widgets)
#     print (len(possible_header_widgets))

print ('possible_header_widgets found')
print (len(possible_header_widgets))
print (list(map(lambda w: [w['itemID'], w['widget']['id']], possible_header_widgets)))

Skipping item because it has no widgets. {'version': 47, 'authoringApp': 'ArcGIS Dashboards', 'authoringAppVersion': '4.23.0+50fddda147'}
Skipping item because it has no widgets. {'version': 41}
Skipping item because it has no widgets. {}
possible_header_widgets found
8
[['8cd84310ce3b45f2abde10c3c9b66896', '3fd99779-524f-45d0-8110-7259dcb1b5b0'], ['2d3ba7964acc415fbd2ad3a72be9f4ce', '4cf0b355-eece-418c-bc2f-350ae0718932'], ['61bcf5ea3fec470d9a5d09d88862e1e7', '4cf0b355-eece-418c-bc2f-350ae0718932'], ['4cabd1d7b951453a8cf1186d948ae225', '4cf0b355-eece-418c-bc2f-350ae0718932'], ['5bc5ee73d5874951b5e9f450ace743d0', '4cf0b355-eece-418c-bc2f-350ae0718932'], ['7df3fc00c65c408a868e860544f22545', '3fd99779-524f-45d0-8110-7259dcb1b5b0'], ['2ba53c5720ec44c39a1cf39ee06f068d', '4cf0b355-eece-418c-bc2f-350ae0718932'], ['6a13139b84ae4a978748356fa81b4aab', '3f4ce5a9-4b7f-45eb-979b-b65a7f5fa47f']]


In [98]:
# check possible_header_widgets texts and make educated guess if it might serve as a header

import sys

# skip this cell if we already have a header
if len(possible_header_widgets) == 0:
    sys.exit('Skipping OpenAI attempt because we have no pre-filtered widgets.')

# try openai to find a header-like widget
from openai import OpenAI
import json
import os

file_path = 'secrets.json'

if os.path.exists(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            secrets = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
else:
    print(f"The file {file_path} does not exist.")

client = OpenAI(
    # This is the default and can be omitted
    api_key=secrets['openai']['apikey'],
)

use_widget = list(filter(lambda x: x['itemID'] == '61bcf5ea3fec470d9a5d09d88862e1e7', possible_header_widgets))[0]
prompt = "Scan this text and omit all empty elements, only keeping elements with less texts, larger fonts and images: {0}".format(use_widget['widget']['text'][0:1000])

print(prompt)

# Use openai.Completion.create to generate completions
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    model="gpt-3.5-turbo",
)
print('response:', response)
    

# # make openai look at all widgets and find one without charts or maps, but text and large fonts
# prompt = "Find a widget that looks like a header. Indicators could be that its text contains large fonts or formatting that indicates that it's being used as a header:\n\n"    
#     prompt += "'Widget {0} out of item {1} contains the following text: '{2}'\n".format(w['widget']['id'], w['itemID'], w['widget']["text"])
# prompt += "Answer:"

# print (prompt[0:1000000])

Scan this text and omit all empty elements, only keeping elements with less texts, larger fonts and images: <h1>Digitaler Zwilling <span style="color:#00a9e6"><strong>Bonn.</strong></span></h1>

<h2>Live-Verkehrs-Daten</h2>

<p>&nbsp;</p>

<p><strong>Eine Auswahl an Offenen Daten zum Thema Verkehr</strong></p>

<p>&nbsp;</p>

<p>&quot;Digitaler Zwilling Bonn&quot;&nbsp;stellt aktuell eine exemplarische Nutzung der Daten dar.</p>

<p>&nbsp;</p>

<p>Weitere Informationen:&nbsp; <a href="https://esri-de.maps.arcgis.com/home/item.html?id=5123bec941b64264b5d5fa94461fafab">Beschreibung</a></p>

<p>&nbsp;</p>

<p>Ein Beispiel der Esri Deutschland GmbH</p>
<svg alt="" height="20" id="gnav-dist-esri-Deutschland-tm" style="transform: rotate(360deg);" version="1.1" viewbox="0 0 208.307 30.001" width="234" x="0px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" y="0px"> <g> <path d="M77.377,0.695c-1.389-0.021-2.533,1.089-2.553,2.478c0,0.003,0,0.00

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [50]:
# send prompt to openAI

import sys

# skip this cell if we haven't generated a prompt
if prompt is None:
    sys.exit('Skipping OpenAI attempt because we haven\'t generated a prompt.')

# Use openai.Completion.create to generate completions
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
)
print(response)

# Access the generated text from the response
generated_text = response['choices'][0]['text']

# Print or use the generated text as needed
print(generated_text)

SystemExit: Skipping OpenAI attempt because we haven't generated a prompt.

In [72]:
# Search template widget only / no OpenAI

import sys

# skip this cell if we already have a header
if header is not None:
    sys.exit('Skipping OpenAI attempt because we already have a header.')
    

item_json = template_json
unwanted_widget_types = ['mapWidget', 'indicatorWidget', 'listWidget', 'embeddedContentWidget', 'serialChartWidget', 'pieChartWidget', 'gaugeWidget', 'legendWidget', 'featureDetailsWidget', 'queryWidget']
widgets = None
if 'widgets' in item_json:
    widgets = item_json['widgets']
if 'desktopView' in item_json:
    widgets = item_json['desktopView']['widgets']
if widgets == None:
    sys.exit('Template item has no widgets.')

filtered_widgets = filter(lambda x: x["type"] not in unwanted_widget_types, widgets)
print (len(filtered_widgets))
print (filtered_widgets)

# # skip this cell if template has no widgets
# if 'widgets' not in template_json:
#     sys.exit('Skipping OpenAI attempt because template has no widgets.')

# # use openai lib to analyze JSON to find all widgets that might serve as a kind of header
# widgets = filter(lambda x: x["type"] not in unwanted_widget_types, template_json['widgets'])

# # make openai look at all widgets and find one without charts or maps, but text and large fonts
# prompt = "Find a widget that looks like a header:\n\n"
# for widget in widgets:
#     prompt += widget["type"] + ":\n"
#     prompt += json.dumps(widget, indent=4) + "\n\n"
# prompt += "Answer:"

# print (prompt)

TypeError: object of type 'filter' has no len()

In [55]:
# Search template widget only / with OpenAI

import sys

# skip this cell if we already have a header
if header is not None:
    sys.exit('Skipping OpenAI attempt because we already have a header.')

# skip this cell if template has no widgets
if 'widgets' not in template_json:
    sys.exit('Skipping OpenAI attempt because template has no widgets.')

# otherwise try openai to find a header-like widget
from openai import OpenAI
import json
import os

file_path = 'secrets.json'

if os.path.exists(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            secrets = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
else:
    print(f"The file {file_path} does not exist.")

client = OpenAI(
    # This is the default and can be omitted
    api_key=secrets['openai']['apikey'],
)
# openai.api_key = secrets['openai']['apikey']

# use openai lib to analyze JSON to find all widgets that might serve as a kind of header
unwanted_widget_types = ['mapWidget', 'indicatorWidget', 'listWidget', 'embeddedContentWidget']
widgets = filter(lambda x: x["type"] not in unwanted_widget_types, template_json['widgets'])

# make openai look at all widgets and find one without charts or maps, but text and large fonts
prompt = "Find a widget that looks like a header:\n\n"
for widget in widgets:
    prompt += widget["type"] + ":\n"
    prompt += json.dumps(widget, indent=4) + "\n\n"
prompt += "Answer:"

print (prompt)

Find a widget that looks like a header:

richTextWidget:
{
    "id": "3f4ce5a9-4b7f-45eb-979b-b65a7f5fa47f",
    "name": "Rich Text (1)",
    "showLastUpdate": false,
    "noDataState": {
        "verticalAlignment": "middle",
        "showCaption": true,
        "showDescription": true
    },
    "noFilterState": {
        "verticalAlignment": "middle",
        "showCaption": true,
        "showDescription": true
    },
    "type": "richTextWidget",
    "text": "<h1>Digitaler Zwilling <span style=\"color:#00c5ff\">Bonn.</span></h1>\n\n<h3>Verkehrsdashboard mit Live Daten</h3>\n\n<p>&nbsp;</p>\n\n<p><strong>Eine Auswahl an offenen Daten zum Thema Verkehr</strong></p>\n\n<p>&quot;Digitaler Zwilling Bonn&quot; stellt eine exemplarische Nutzung der Daten dar.</p>\n\n<p>&nbsp;</p>\n\n<p>Weitere Informationen: <a href=\"https://esri-de.maps.arcgis.com/home/item.html?id=5637facbb2c04bea96b31c217e048c99\">Beschreibung</a></p>\n"
}

Answer:


In [50]:
import sys

# skip this cell if we haven't generated a prompt
if prompt is None:
    sys.exit('Skipping OpenAI attempt because we haven\'t generated a prompt.')

# Use openai.Completion.create to generate completions
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
)
print(response)

# Access the generated text from the response
generated_text = response['choices'][0]['text']

# Print or use the generated text as needed
print(generated_text)

SystemExit: Skipping OpenAI attempt because we haven't generated a prompt.

### Apply template header and theme

In [ ]:
for item in moved_items:
    # get template dashboard JSON
    edit_json = item.get_data()

    # copy header section from template_json
    if edit_json["version"] > 57:
        # TODO: check since when we have a "desktopView"
        edit_json["desktopView"]["header"] = header
    else:
        edit_json["header"] = header

    # copy theme property from template_json
    edit_json["theme"] = theme

    # copy themeOverrides from template_json
    edit_json["themeOverrides"] = themeOverrides
 
    # store item to ArcGIS Online or ArcGIS Enterprise
    update_success = item.update(data=edit_json)
    
    if update_success:
        print ("Item " + item.title + " (" + item.id + ") updated.")
    else:
        sys.exit("ERROR: Item {0} ({1}) could not be updated. ".format(item.title, item.id))